In [ ]:
from pyspark import SparkConf, SparkContext

In [ ]:
sc = SparkContext(master="yarn")

In [ ]:
sc

In [ ]:
def computeContribs(neighbours, pagerank):
    num = len(neighbours)
    
    for n in neighbours:
        yield (n, pagerank / num)

In [ ]:
iterations = 1
raw_graph = sc.textFile("/data/spark/lecture05/graph.tsv")
graph = raw_graph.map(lambda x: tuple(x.split("\t")))\
                 .distinct()
vertices = graph.map(lambda x: x[0])\
                .union(graph.map(lambda x: x[1]))\
                .distinct()
num_vertices = vertices.count()
ranks = vertices.map(lambda x: (x, 1 / num_vertices))

for i in range(iterations):
    links = graph.groupByKey()
    contributions = links.join(ranks)
    ranks = contributions.flatMap(lambda x: computeContribs(x[1][0], x[1][1]))\
                         .groupByKey().mapValues(sum)
    print("Iteration {0}: current pagerank {1}".format(i, sorted(ranks.collect(), key=lambda x: x[0])))

## Попытайтесь отпимизировать эту программу (сократить число стеджей, тасков и объем шаффла), используя знания, полученные на занятиях

In [ ]:
## Ваш код здесь

In [ ]:
sc.stop()